## Exploring the SciQ dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-'

#### Install the autolabel library

In [2]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [2]:
from autolabel import get_data

get_data('belebele')

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [3]:
import json

from autolabel import LabelingAgent

In [4]:
# load the config
with open('config_belebele.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `question_answering` (since it's a question answering task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at answer science questions...` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 10 (how many labeled examples to provide to the LLM)

In [5]:
config

{'task_name': 'QuestionAnsweringTrivia',
 'task_type': 'question_answering',
 'dataset': {'label_column': 'answer', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at answering questions based on context. Your job is to answer the given question, using the options provided for each question. Choose the best answer for the question from among the options provided',
  'example_template': 'Question: {question}\nOptions: {options}\nAnswer: {answer}',
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 10}}

In [6]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [7]:
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $3.8817 │
│ Number of Examples       │ 800     │
│ Average cost per example │ $0.0049 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at answering questions based on context. Your job is to answer the given question, using the 
options provided for each question. Choose the best answer for the question from among the options provided

You will return the answer one element: "the correct label"


Some examples with their output answers are provided below:

Question: Context: "Widespread looting reportedly continued overnight, as law enforcement officers were not present
on Bishkek's streets. Bishkek was described as sinking into a state of ""anarchy"" by one observer, as gangs of 
people roamed the streets and plundered stores of consumer goods. Several Bishkek residents blamed protesters from 
the south for the lawlessness." Question: Who did several of Bishkek’s residents think were responsible for the 
looting?
Options: ['Law enforcement officers', 'Store owners', 'Protesters', 'Teenagers']
Answer: Protesters

Question: Context: During his trip, Iwasaki ran into trouble on many occasions. He was robbed by pirates, attacked 
in Tibet by a rabid dog, escaped marriage in Nepal and was arrested in India. Question: Who attacked Iwasaki?
Options: ['His potential spouse', 'Pirates', 'A dog', 'Police']
Answer: A dog

Question: Context: Organisers of the protest said about 100,000 people turned up in German cities such as Berlin, 
Cologne, Hamburg, and Hanover. In Berlin, police estimated 6,500 protestors. Protests also took place in Paris, 
Sofia in Bulgaria, Vilnius in Lithuania, Valetta in Malta, Tallinn in Estonia, and Edinburgh and Glasgow in 
Scotland. In London, about 200 people protested outside some major copyright holders' offices. Last month, there 
were major protests in Poland when that country signed ACTA, which has led to the Polish government deciding not to
ratify the agreement, for now. Latvia and Slovakia have both delayed the process of joining ACTA. Question: In 
response to the protests, which country did not move forward with their signed ACTA agreement?
Options: ['Germany', 'Poland', 'Scotland', 'Lithuania']
Answer: Poland

Question: Context: "The celebrations started with a special show by the world-renowned group Cirque du Soleil. It 
was followed by the Istanbul State Symphony Orchestra, a Janissary band, and the singers Fatih Erkoç and Müslüm 
Gürses. Then Whirling Dervishes took to the stage. Turkish diva Sezen Aksu performed with the Italian tenor 
Alessandro Safina and Greek singer Haris Alexiou. To finish, Turkish dance group Fire of Anatolia performed the 
show ""Troy""." Question: What was the second-last performance of the evening?
Options: ['Sezen Aksu, Alessandro Safina and Haris Alexiou', 'Fire of Anatolia', 'Whirling Dervishes', 'Fatih Erkoç
and Müslüm Gürses']
Answer: Sezen Aksu, Alessandro Safina and Haris Alexiou

Question: Context: The terrified King Louis XVI, Queen Marie Antoinette their two young children (11 year old Marie
Therese and four year old Louis-Charles) and the King's sister, Madam Elizabeth, on the 6th October 1789 were 
forced back to Paris from Versailles by a mob of market women. In a carriage, they traveled back to Paris 
surrounded by a mob of people screaming and shouting threats against the King and Queen. The mob of people forced 
the King And Queen to have their carriage windows wide open. At one point a member of the mob waved the head of a 
royal guard killed at Versailles in front of the terrified Queen. Question: Which of the following was not forced 
upon the King and the Queen by the mob?
Options: ['The opening of their carriage windows', 'An abrupt exit from Versailles ', 'A viewing of the guard’s 
beheading', 'A trip back to Paris']
Answer: A viewing of the guard’s beheading

Question: Context: Martelly swore in a new Provisional Electoral Council (CEP) of nine members yesterday. It is 
Martelly's fifth CEP in four years. Last month a presidential commission recommended the prior CEP's resignation as
part of a package of measures to move the country towards new elections. T

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:
ds = agent.run(ds, max_items=100)

Output()

2023-10-04 21:08:31 openai INFO: error_code=rate_limit_exceeded error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 87436 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-10-04 21:08:31 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 87436 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-10-04 21:08:33 openai INFO: error_code=rate_limit_exceeded error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkY

Actual Cost: 0.2864

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ f1     ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0.86     │ 100     │ 1.0             │ 0.8842 │
└──────────┴─────────┴─────────────────┴────────┘